In [1]:
# Imports
import pandas as pd
import durak
from AttackField import AttackField
from mesa import Model
from CustomStagedActivation import CustomStagedActivation
from Player import Player
from Deck import Deck
from DiscardPile import DiscardPile
import random
from newKripke import *
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import copy

In [2]:
def play_multiple(
    num_games,
    verbose = False,
    num_players = 3,
    num_suits = 3,
    num_cards_per_suit = 3,
    num_starting_cards = 2,
    player_strategies = ["normal", "normal", "normal"],
    player_depths = [1,1,1],
    ):
    
    multiple = {}
    data = {}

    # player_strategies = player_strategies
    # player_depths = player_depths
    num_players = num_players
    num_suits = num_suits
    num_cards_per_suit = num_cards_per_suit
    num_starting_cards = num_starting_cards

    # Put everything in the multiple dictionary
    multiple["player_strategies"] = player_strategies
    multiple["player_depths"] = player_depths
    multiple["num_players"] = num_players
    multiple["num_suits"] = num_suits
    multiple["num_cards_per_suit"] = num_cards_per_suit
    multiple["num_starting_cards"] = num_starting_cards

    original_model = durak.DurakModel(
        multiple = multiple,
        verbose = verbose, 
        num_suits = num_suits, 
        num_cards_per_suit = num_cards_per_suit, 
        num_starting_cards = num_starting_cards,
        player_strategies = player_strategies,
        player_depths = player_depths,
        multiple_runs = True
        )

    # Create the initial Kripke model with all players and all cards in the deck
    original_model.kripke_deck = [str(c) for c in original_model.deck.deck]
    original_model.kripke_discard_pile = [str(c) for c in original_model.discard_pile.cards]
    original_model.kripke_players = [str(p.get_id()) for p in original_model.players]
    original_model.kripke_card_locations = ["Deck", "Discard"]
    original_model.kripke_card_locations.extend(original_model.kripke_players)
    original_model.kripke_worlds = gen_worlds(original_model.kripke_deck, original_model.kripke_card_locations, original_model.kripke_players)
    original_model.kripke_model, original_model.reachable_worlds = gen_empty_kripke(original_model.kripke_worlds, original_model.kripke_players)

    for game_index in range(num_games):
        print("=================== NEW GAME STARTED ===================")
        print("=================== NEW GAME STARTED ===================")
        print("=================== NEW GAME STARTED ===================")

        model = copy.deepcopy(original_model)

        # Deal
        for i in range(model.num_starting_cards):
            for player in model.players:
                player.receive_card(model.deck.deal())

        # Select a random starting attacker and set the defender
        model.current_attacker = random.choice(model.players)
        model.current_defender = model.current_attacker.get_next_player()

        # Add the starting card knowledge to the Kripke model
        for kripke_player in model.kripke_players:
            for card in player.hand.get_cards_in_hand():
                statement = Atom(kripke_player + str(card))
                add_links(model.kripke_model, kripke_player, statement, model.reachable_worlds)

        data["game" + str(game_index)] = durak.play(model)


    return data

In [3]:
# Experiment 1
num_runs = 3
# --> Test the normal strategy at depth 1 against random strategies at a depth of 1
player_strategies = ["normal", "random", "random"]
player_depths = [1,1,1]
data = play_multiple(num_runs, player_strategies=player_strategies, player_depths=player_depths, verbose= False)
df = pd.DataFrame.from_dict(data, orient="index")
df.to_csv("d1_normal-d1_random-d1_random.csv")

# Experiment 2
num_runs = 2
# --> Test the normal strategy at depth 2 against random strategies at a depth of 1
player_strategies = ["normal", "random", "random"]
player_depths = [2,1,1]
data = play_multiple(num_runs, verbose= False, player_strategies=player_strategies, player_depths=player_depths)
df = pd.DataFrame.from_dict(data, orient="index")
df.to_csv("d2_normal-d1_random-d1_random.csv")

# Experiment 3
num_runs = 2
# --> Test the normal strategy at depth2 against normal strategies of depth 1
player_strategies = ["normal", "normal", "normal"]
player_depths = [2,1,1]
data = play_multiple(num_runs, verbose= False, player_strategies=player_strategies, player_depths=player_depths)
df = pd.DataFrame.from_dict(data, orient="index")
df.to_csv("d2_normal-d1_normal-d1_normal.csv")

# Experiment 4
num_runs = 2
# --> Test the optimal strategies at depoth 2 against each other to get a baseline
player_strategies = ["normal", "normal", "normal"]
player_depths = [2,2,2]
data = play_multiple(num_runs, verbose= False, player_strategies=player_strategies, player_depths=player_depths)
df = pd.DataFrame.from_dict(data, orient="index")
df.to_csv("d2_normal-d2_normal-d2_normal.csv")

# Experiment 5
# --> Test the influence of additional cards on the value of the normal strategy at depth 2
# num_total_cards = [] 
# columns = ["num_cards"]
# players = sorted(list(df.durak.unique()))
# columns.extend(players)
# wins_per_num_cards = []
# total_df = pd.DataFrame()
# num_suits = 3

# for num in range(3,5):
#     # num_total_cards.append(num)
#     data = play_multiple(1, verbose= False, num_cards_per_suit=num, num_suits=num_suits)
#     df = pd.DataFrame.from_dict(data, orient="index")
#     total_df.append(df)
    # win_percentages = [100 - (x/len(df)*100) for x in list(Counter(df["durak"]).values())]
    # li = [num_suits * num]
    # li.extend(win_percentages)
    # wins_per_num_cards.append(li)
    # print(li)

# wins_per_num_cards

# df = pd.DataFrame(wins_per_num_cards, columns=columns)
total_df.to_csv("d2_normal-d1_random-d1_random_more_cards.csv")

Generated 965853 worlds.
Generated empty model.
=================== NEW GAME STARTED ===================
=================== NEW GAME STARTED ===================
=================== NEW GAME STARTED ===================
Adding links...
	 Number of worlds from/to which to add relations: 196271
Adding links...
	 Number of worlds from/to which to add relations: 196271
Adding links...
	 Number of worlds from/to which to add relations: 196271
Adding links...
	 Number of worlds from/to which to add relations: 196271
Adding links...
	 Number of worlds from/to which to add relations: 196271
Adding links...
	 Number of worlds from/to which to add relations: 196271
Removing links...
	 Number of worlds from/to which to remove relations (incl. already unreachable): 769582
Removing links...
	 Number of worlds from/to which to remove relations (incl. already unreachable): 775565
Removing links...
	 Number of worlds from/to which to remove relations (incl. already unreachable): 775565
Removing links..

UnboundLocalError: local variable 'card' referenced before assignment

In [ ]:
"""
Percentage of losses per player
"""

fig, ax = plt.subplots()
durak_percentages = [x/len(df)*100 for x in list(Counter(df["durak"]).values())]
players = sorted(list(df.durak.unique()))

g = sns.barplot(x=players, y=durak_percentages, dodge=False)
g.set_xlabel("Player", fontsize=18)
g.set_xticklabels(["Player " + str(x) for x in players])

g.set_ylabel("Percentage of losses", fontsize=18)

ax=g
#annotate axis = seaborn axis
for p in ax.patches:
    ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=14, color='black', xytext=(0, 20),
                textcoords='offset points')
_ = g.set_ylim(0, 100) #To make space for the annotations

In [5]:
"""
Influence of total number of cards on the effectiveness of depth
"""

num_total_cards = [] 
columns = ["num_cards"]
players = sorted(list(df.durak.unique()))
columns.extend(players)
wins_per_num_cards = []
num_suits = 3

# We keep it at 3 suits (?), but loop over the number of cards per suit
for num in range(3,14):

    num_total_cards.append(num)

    data = play_multiple(10, verbose= False, num_cards_per_suit=num, num_suits=num_suits)
    df = pd.DataFrame.from_dict(data, orient="index")
    win_percentages = [100 - (x/len(df)*100) for x in list(Counter(df["durak"]).values())]

    li = [num_suits * num]
    li.extend(win_percentages)
    wins_per_num_cards.append(li)
    print(li)

wins_per_num_cards

df2 = pd.DataFrame(wins_per_num_cards, columns=columns)
df2

sns.lineplot(x="num_cards", y="value",
             hue="variable",
             data = pd.melt(df2, ["num_cards"]))

Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 worlds.
Generated 19683 

KeyboardInterrupt: 